In [7]:
import pandas as pd
import os
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

# ----------------------------------------
# 1. Cargar datos históricos de Primera División
carpeta = 'C:/Users/laztr/OneDrive/Escritorio/IA fútbol/'
archivos = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.csv')]
dataframes = []

for archivo in archivos:
    df_temp = pd.read_csv(archivo)
    df_temp.rename(columns={
        'Date': 'Fecha',
        'HomeTeam': 'Local',
        'AwayTeam': 'Visitante',
        'FTHG': 'Goles_Local',
        'FTAG': 'Goles_Visitante'
    }, inplace=True)
    dataframes.append(df_temp[['Fecha', 'Local', 'Visitante', 'Goles_Local', 'Goles_Visitante']])

df = pd.concat(dataframes, ignore_index=True)

# ----------------------------------------
# 1b. Cargar datos del Oviedo
carpeta_oviedo = "C:/Users/laztr/OneDrive/Escritorio/Real Oviedo/"
archivos_oviedo = [os.path.join(carpeta_oviedo, f) for f in os.listdir(carpeta_oviedo) if f.endswith('.csv')]
partidos_oviedo = []

for archivo in archivos_oviedo:
    df_oviedo_temp = pd.read_csv(archivo)
    df_oviedo_temp.columns = [col.strip() for col in df_oviedo_temp.columns]
    columnas = {'Date': 'Fecha', 'HomeTeam': 'Local', 'AwayTeam': 'Visitante',
                'FTHG': 'Goles_Local', 'FTAG': 'Goles_Visitante'}
    df_oviedo_temp.rename(columns=columnas, inplace=True)
    necesarias = ['Fecha', 'Local', 'Visitante', 'Goles_Local', 'Goles_Visitante']
    if all(col in df_oviedo_temp.columns for col in necesarias):
        df_oviedo_temp = df_oviedo_temp[necesarias]
        partidos_oviedo.append(df_oviedo_temp)

df_oviedo = pd.concat(partidos_oviedo, ignore_index=True)

# ----------------------------------------
# 1c. Combinar datasets
df = pd.concat([df, df_oviedo], ignore_index=True)


df = df.dropna(subset=['Fecha', 'Local', 'Visitante'])
df = df.drop_duplicates(subset=['Fecha', 'Local', 'Visitante'], keep='first')

# ----------------------------------------
# 2. 🔤 Normalizar nombres de equipos 
df['Local'] = df['Local'].astype(str).str.strip()
df['Visitante'] = df['Visitante'].astype(str).str.strip()

# Cambios de nombres para presentación
nombres_bonitos = {
    "Ath Madrid": "Atlético de Madrid",
    "Alaves": "Alavés",
    "Ath Bilbao": "Athletic Club",
    "Espanol": "Espanyol",
    "Sociedad": "Real Sociedad",
    "Vallecano": "Rayo Vallecano"
}

df['Local'] = df['Local'].replace(nombres_bonitos)
df['Visitante'] = df['Visitante'].replace(nombres_bonitos)

# Normalización adicional del Real Oviedo 
reemplazos_oviedo = {
    'Oviedo': 'Real Oviedo',
    'Real Oviedo SAD': 'Real Oviedo',
    'R. Oviedo': 'Real Oviedo',
    'Real_Oviedo': 'Real Oviedo'
}
df['Local'] = df['Local'].replace(reemplazos_oviedo)
df['Visitante'] = df['Visitante'].replace(reemplazos_oviedo)


# ----------------------------------------
# 3. 📅 Convertir fechas y obtener años 
import datetime as dt

def parse_fecha_segura(fecha_str):
    if pd.isna(fecha_str):
        return pd.NaT
    fecha_str = str(fecha_str).strip()
    try:
        # Intentar primero con formato d/m/a
        fecha = pd.to_datetime(fecha_str, format="%d/%m/%y", dayfirst=True)
        # Corrige años de 2 dígitos (ej: 09 -> 2009, no 1909 ni 2029)
        if fecha.year > dt.datetime.now().year + 1:  # si parece del futuro
            fecha = fecha - pd.DateOffset(years=100)
        return fecha
    except Exception:
        try:
            # Reintentar con formato d/m/yyyy
            return pd.to_datetime(fecha_str, format="%d/%m/%Y", dayfirst=True)
        except Exception:
            return pd.NaT

df['Fecha'] = df['Fecha'].apply(parse_fecha_segura)
df = df.dropna(subset=['Fecha'])
df['Año'] = df['Fecha'].dt.year


# ----------------------------------------
# 4. ⚽ Filtrar SOLO equipos de Primera División
equipos_primera = [
    "Alavés", "Athletic Club", "Atlético de Madrid", "Barcelona", "Betis", "Celta",
    "Elche", "Espanyol", "Getafe", "Girona", "Levante", "Mallorca",
    "Osasuna", "Real Madrid", "Sevilla", "Real Sociedad", "Valencia",
    "Rayo Vallecano", "Villarreal", "Real Oviedo"
]
df = df[df['Local'].isin(equipos_primera) & df['Visitante'].isin(equipos_primera)]

# ----------------------------------------
# 5. Crear columna Resultado
def obtener_resultado(row):
    if row['Goles_Local'] > row['Goles_Visitante']:
        return 'Local'
    elif row['Goles_Local'] < row['Goles_Visitante']:
        return 'Visitante'
    else:
        return 'Empate'

df['Resultado'] = df.apply(obtener_resultado, axis=1)

# ----------------------------------------
# 6. Calcular puntos
def puntos(row):
    if row['Resultado'] == 'Local':
        return 3, 0
    elif row['Resultado'] == 'Visitante':
        return 0, 3
    else:
        return 1, 1

df[['Puntos_Local', 'Puntos_Visitante']] = df.apply(puntos, axis=1, result_type='expand')

# ----------------------------------------
# 7. 📈 Calcular forma reciente (últimos 5 partidos por equipo)
df = df.sort_values('Fecha')

def calcular_forma(df, equipo_col, goles_favor_col, goles_contra_col, puntos_col, prefix):
    resultados = []
    for equipo, partidos in df.groupby(equipo_col):
        partidos = partidos.sort_values('Fecha')
        goles_favor_rolling = partidos[goles_favor_col].shift(1).rolling(5, min_periods=1).mean()
        goles_contra_rolling = partidos[goles_contra_col].shift(1).rolling(5, min_periods=1).mean()
        puntos_rolling = partidos[puntos_col].shift(1).rolling(5, min_periods=1).mean()
        resultados.append(pd.DataFrame({
            prefix + '_Forma_Goles': goles_favor_rolling,
            prefix + '_Forma_Concedidos': goles_contra_rolling,
            prefix + '_Forma_Puntos': puntos_rolling
        }, index=partidos.index))
    return pd.concat(resultados)

forma_local = calcular_forma(df, 'Local', 'Goles_Local', 'Goles_Visitante', 'Puntos_Local', 'Local')
forma_visitante = calcular_forma(df, 'Visitante', 'Goles_Visitante', 'Goles_Local', 'Puntos_Visitante', 'Visitante')

df = pd.concat([df, forma_local, forma_visitante], axis=1)
# 🕵️‍♂️ Verificar la "forma reciente" de un equipo concreto (por ejemplo, Barcelona)

equipo_test = "Barcelona"

# Filtramos partidos donde participa el equipo (como local o visitante)
partidos_equipo = df[(df["Local"] == equipo_test) | (df["Visitante"] == equipo_test)].copy()
partidos_equipo = partidos_equipo.sort_values("Fecha", ascending=False)



# Revisa qué 5 partidos usa la IA para calcular la forma reciente
partidos_equipo_orden = partidos_equipo.sort_values("Fecha")

# Selecciona el bloque de los últimos 5 partidos más recientes
ultimos_5 = partidos_equipo_orden.tail(5)

# ----------------------------------------
# 8. 📊 Estadísticas base + diferencias
prom_goles_local = df.groupby('Local')['Goles_Local'].mean().to_dict()
prom_goles_visitante = df.groupby('Visitante')['Goles_Visitante'].mean().to_dict()
prom_puntos_local = df.groupby('Local')['Puntos_Local'].mean().to_dict()
prom_puntos_visitante = df.groupby('Visitante')['Puntos_Visitante'].mean().to_dict()

df['Prom_Goles_Local'] = df['Local'].map(prom_goles_local)
df['Prom_Goles_Visitante'] = df['Visitante'].map(prom_goles_visitante)
df['Fuerza_Local'] = df['Local'].map(prom_puntos_local)
df['Fuerza_Visitante'] = df['Visitante'].map(prom_puntos_visitante)

df['dif_goles'] = df['Prom_Goles_Local'] - df['Prom_Goles_Visitante']
df['dif_fuerza'] = df['Fuerza_Local'] - df['Fuerza_Visitante']

# ----------------------------------------
# 9. Codificar resultado y ponderar temporadas recientes
df['Resultado_cod'] = df['Resultado'].map({'Local': 0, 'Empate': 1, 'Visitante': 2})
peso_por_año = {2025: 3.0, 2024: 2.5, 2023: 2.0, 2022: 1.5, 2021: 1.2}
df['Peso'] = df['Año'].map(peso_por_año).fillna(1.0)


# ----------------------------------------
# 10. ENTRENAMIENTO DE MODELOS HÍBRIDOS 
features = [
    'Prom_Goles_Local', 'Prom_Goles_Visitante', 'Fuerza_Local', 'Fuerza_Visitante',
    'dif_goles', 'dif_fuerza',
    'Local_Forma_Goles', 'Local_Forma_Concedidos', 'Local_Forma_Puntos',
    'Visitante_Forma_Goles', 'Visitante_Forma_Concedidos', 'Visitante_Forma_Puntos'
]

df = df.dropna(subset=features)
X = df[features]
y = df['Resultado_cod']
w = df['Peso']

X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y, w, stratify=y, random_state=42, test_size=0.2
)

# Modelos base
modelo_xgb = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False, random_state=42)
modelo_rf = RandomForestClassifier(n_estimators=300, random_state=42, max_depth=10, class_weight='balanced')
modelo_lr = LogisticRegression(max_iter=2000, multi_class='multinomial')

# Entrenamiento
modelo_xgb.fit(X_train, y_train, sample_weight=w_train)
modelo_rf.fit(X_train, y_train)
modelo_lr.fit(X_train, y_train, sample_weight=w_train)

# ----------------------------------------
# 11. 🔮 Ensemble (voto ponderado)
pred_xgb = modelo_xgb.predict_proba(X_test)
pred_rf = modelo_rf.predict_proba(X_test)
pred_lr = modelo_lr.predict_proba(X_test)

peso_xgb, peso_rf, peso_lr = 0.1, 0.5, 0.4
pred_ensemble = (pred_xgb * peso_xgb) + (pred_rf * peso_rf) + (pred_lr * peso_lr)
y_pred_final = np.argmax(pred_ensemble, axis=1)

# ----------------------------------------
# 13. 💾 Guardar modelo
joblib.dump({
    'modelo_xgb': modelo_xgb,
    'modelo_rf': modelo_rf,
    'modelo_lr': modelo_lr,
    'prom_goles_local': prom_goles_local,
    'prom_goles_visitante': prom_goles_visitante,
    'prom_puntos_local': prom_puntos_local,
    'prom_puntos_visitante': prom_puntos_visitante,
    'pesos': (peso_xgb, peso_rf, peso_lr)
}, 'modelo_hibrido_laliga.pkl')

# 14. 🔮 FUNCIÓN DE PREDICCIÓN DE PARTIDOS ESPECÍFICOS
def predecir_partido(local, visitante):
    """
    Predice el resultado más probable de un partido entre dos equipos.
    Usa el modelo híbrido (XGB + RF + LR) entrenado previamente.
    """
    # Verificar que ambos equipos existen
    if local not in equipos_primera or visitante not in equipos_primera:
        print(f"⚠️ Uno de los equipos no pertenece a Primera División: {local}, {visitante}")
        return None

    if local == visitante:
        print("⚠️ No se pueden enfrentar el mismo equipo.")
        return None

    # Obtener medias y forma reciente del dataset
    try:
        datos_local = df[df['Local'] == local].iloc[-1]
        datos_visitante = df[df['Visitante'] == visitante].iloc[-1]
    except IndexError:
        print("⚠️ No hay datos suficientes para uno de los equipos.")
        return None

    # Crear el vector de características
    X_pred = pd.DataFrame([{
        'Prom_Goles_Local': prom_goles_local.get(local, 1.0),
        'Prom_Goles_Visitante': prom_goles_visitante.get(visitante, 1.0),
        'Fuerza_Local': prom_puntos_local.get(local, 1.0),
        'Fuerza_Visitante': prom_puntos_visitante.get(visitante, 1.0),
        'dif_goles': prom_goles_local.get(local, 0) - prom_goles_visitante.get(visitante, 0),
        'dif_fuerza': prom_puntos_local.get(local, 0) - prom_puntos_visitante.get(visitante, 0),
        'Local_Forma_Goles': datos_local['Local_Forma_Goles'],
        'Local_Forma_Concedidos': datos_local['Local_Forma_Concedidos'],
        'Local_Forma_Puntos': datos_local['Local_Forma_Puntos'],
        'Visitante_Forma_Goles': datos_visitante['Visitante_Forma_Goles'],
        'Visitante_Forma_Concedidos': datos_visitante['Visitante_Forma_Concedidos'],
        'Visitante_Forma_Puntos': datos_visitante['Visitante_Forma_Puntos']
    }])


    # Obtener probabilidades de cada modelo
    p_xgb = modelo_xgb.predict_proba(X_pred)
    p_rf = modelo_rf.predict_proba(X_pred)
    p_lr = modelo_lr.predict_proba(X_pred)

    # Voto ponderado
    p_final = (p_xgb * peso_xgb) + (p_rf * peso_rf) + (p_lr * peso_lr)
    pred = np.argmax(p_final, axis=1)[0]

    etiquetas = ['Local', 'Empate', 'Visitante']
    resultado = etiquetas[pred]
    probas = p_final[0]

    print(f"\n⚔️  Predicción del partido: {local} vs {visitante}")
    print(f"📊  Probabilidades -> Local: {probas[0]:.2%}, Empate: {probas[1]:.2%}, Visitante: {probas[2]:.2%}")
    print(f"✅  Resultado más probable: {resultado}")
    return resultado, probas

predecir_partido('Barcelona','Getafe')



⚔️  Predicción del partido: Barcelona vs Getafe
📊  Probabilidades -> Local: 86.51%, Empate: 9.43%, Visitante: 4.06%
✅  Resultado más probable: Local


('Local', array([0.86506936, 0.09433283, 0.04059782]))